# Clean Demographic Data
#### Daniel Fay

* % population 16 and Under
* % population 65+
* % white
* % black
* % other

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
demo_data = gpd.read_file('../Data/Tract_2010Census_DP1/Tract_2010Census_DP_Cuyahoga.shp')
econ_data = pd.read_table('../Data/econ_data_tab.txt')

In [3]:
demo_data.head()

,ALAND10,AWATER10,DP0010001,DP0010002,DP0010003,DP0010004,DP0010005,DP0010006,DP0010007,DP0010008,...,DP0220002,DP0230001,DP0230002,GEOID10,INTPTLAT10,INTPTLON10,NAMELSAD10,Shape_Area,Shape_Leng,geometry
0,6449964.0,0.0,3788,226,313,339,313,154,119,202,...,183,3.02,2.95,39103415100,+41.2586108,-081.7942495,Census Tract 4151,0.000693,0.111576,POLYGON ((-81.80648576188759 41.27607673330996...
1,8913134.0,0.0,6651,569,530,446,384,247,463,553,...,840,2.80,1.88,39103415800,+41.2603471,-081.8632294,Census Tract 4158,0.000958,0.155166,POLYGON ((-81.85016099999996 41.27542499999998...
2,1922712.0,0.0,2602,154,182,198,209,127,137,135,...,305,2.79,2.16,39103416300,+41.2724689,-081.8210791,Census Tract 4163,0.000207,0.076191,"(POLYGON ((-81.835014 41.27560900000009, -81.8..."
3,69162378.0,360814.0,7646,376,486,619,520,323,225,272,...,382,2.78,2.60,39103400100,+41.2490837,-081.7378664,Census Tract 4001,0.007468,0.349380,(POLYGON ((-81.78509684929675 41.2764304301054...
4,17565807.0,204826.0,4429,223,228,230,222,266,350,252,...,1838,2.46,1.95,39085201100,+41.5954183,-081.4087971,Census Tract 2011,0.001919,0.231141,(POLYGON ((-81.41064907299379 41.5697726288051...


In [4]:
demo_data['Under_16_Percent'] = (demo_data['DP0010001'] - demo_data['DP0030001'])/demo_data['DP0010001']
demo_data['65_Over_Percent'] = demo_data['DP0070001']/demo_data['DP0010001']
demo_data['White_Percent'] = demo_data['DP0080003']/demo_data['DP0080001']
demo_data['Black_Percent'] = demo_data['DP0080004']/demo_data['DP0080001']
demo_data['Other_Percent'] = 1 - (demo_data['White_Percent'] + demo_data['Black_Percent'])
demo_data = demo_data[['Under_16_Percent', '65_Over_Percent', 'White_Percent', 'Black_Percent', 'Other_Percent', 'DP0010001',\
                      'geometry', 'NAMELSAD10']]
demo_data.head()

,Under_16_Percent,65_Over_Percent,White_Percent,Black_Percent,Other_Percent,DP0010001,geometry,NAMELSAD10
0,0.251584,0.082365,0.960137,0.008712,0.031151,3788,POLYGON ((-81.80648576188759 41.27607673330996...,Census Tract 4151
1,0.245527,0.093670,0.952488,0.009472,0.038039,6651,POLYGON ((-81.85016099999996 41.27542499999998...,Census Tract 4158
2,0.225596,0.116065,0.961952,0.011530,0.026518,2602,"(POLYGON ((-81.835014 41.27560900000009, -81.8...",Census Tract 4163
3,0.207952,0.143735,0.974627,0.004054,0.021318,7646,(POLYGON ((-81.78509684929675 41.2764304301054...,Census Tract 4001
4,0.163694,0.191917,0.816437,0.104087,0.079476,4429,(POLYGON ((-81.41064907299379 41.5697726288051...,Census Tract 2011


In [5]:
econ_tracts = []
for ix, row in econ_data.iterrows():
    ct = row['Geography'].split(',')[0]
    econ_tracts.append(ct)

econ_data['NAMELSAD10'] = econ_tracts

In [6]:
econ_data = econ_data[['NAMELSAD10', \
                       'Estimate; Median household income in the past 12 months (in 2015 Inflation-adjusted dollars)']]
econ_data.columns = ['NAMELSAD10', 'Median_Income']
econ_data.head()

,NAMELSAD10,Median_Income
0,Census Tract 1832,172656
1,Census Tract 1791.02,170568
2,Census Tract 1833,165043
3,Census Tract 1963,154297
4,Census Tract 1791.01,138594


In [7]:
demo_ct = pd.merge(demo_data, econ_data, on='NAMELSAD10')
demo_ct.head()

,Under_16_Percent,65_Over_Percent,White_Percent,Black_Percent,Other_Percent,DP0010001,geometry,NAMELSAD10,Median_Income
0,0.173797,0.131016,0.073864,0.904746,0.021390,2992,POLYGON ((-81.55600099999998 41.54523699999993...,Census Tract 1261,35100
1,0.171733,0.207264,0.954599,0.007501,0.037900,2533,"POLYGON ((-81.87535899999995 41.4176480000001,...",Census Tract 1531.03,63281
2,0.215865,0.156047,0.136541,0.824447,0.039012,1538,POLYGON ((-81.55588699999998 41.53788500000002...,Census Tract 1401,60143
3,0.222466,0.130759,0.043440,0.930671,0.025889,2279,POLYGON ((-81.55663099999998 41.52714300000014...,Census Tract 1403.01,40938
4,0.195163,0.176920,0.327111,0.627917,0.044972,2357,"POLYGON ((-81.55001399999998 41.520512, -81.55...",Census Tract 1403.02,41582


In [8]:
demo_ct.crs

{'init': u'epsg:4269'}

In [9]:
demo_ct.to_file('../Data/processed/shapefiles/CT_Demo.shp')

In [10]:
demo_ct.to_csv("../Data/CT_Demo_Data.csv")